# MNIST With ConvNets
In this notebook, we continue the work of "mnist.ipynb" of understanding Pytorch basics with MNIST. Our aim is to use convnets intead of a simple MLP, so only `Defining a Neural Network` and `Loss Function and Optmizer` sections will be new.

**We will not focus on architecture choices, but on framework implementation and concepts understanding**.

In [1]:

# Imports
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Defines transform pipeline
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0,), (1,))
])

# Download and transform
train_dataset = datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = datasets.MNIST(
    root='./data', train=False, download=True, transform=transform
)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## Defining a Neural Network
A key difference from our previous notebook is that a ConvNet has **convolutional layers**. These layers perform a crucial task called **feature extraction**—the process of capturing various abstract characteristics of an image by "scanning" different small sections of it.

Our model will have 2 major parts:
1. **Convolution**: for feature extraction;
2. **Learning**: for classification/decision making;

### 1. Convolution
Defined with `nn.Conv2d`. In our model, we define 3 convolution layers with different sizes resulting in 1 channel of input (our images are grayscale) and 128 channels of output. At `foward()`, we define that at each layer will be a transformation like **convolution -> ReLU -> pooling**.
- **Convolution:** feature extraction with
    - **1 to 32 channels + ReLU + Pooling:** high resolution features with non-linearity;
    - **32 to 64 channels + ReLU + Pooling:** less resolution features with non-linearity (more specific features);
    - **53 to 128 channels + ReLU + Pooling:** low resolution feaures with non-linearity (general features);
    - `kernel_size=3`: filter 3x3;
    - `padding="same"`: keeps image size;
    - **dropout(25%):** deactivates 25% of the neurons to prevent overfitting;

- **Learning:**
    - **first fully connected layer:** (1152 -> 512) + ReLU
    - **dropout(50%):** deactivates 50% of the neurons to prevent overfitting;
    - **second fully connected layer:** (512 -> 10) + ReLU;
    - **softmax (log):** last layer are classification probabilities

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding="same")
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding="same")
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding="same")
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 3 * 3, 512)
        self.fc2 = nn.Linear(512, 10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # initiates with size = (1, 28, 28)
        x = self.pool(F.relu(self.conv1(x))) # (32, 14, 14)
        x = self.pool(F.relu(self.conv2(x))) # (64, 7, 7)
        x = self.pool(F.relu(self.conv3(x))) # (128, 3, 3)
        x = self.dropout1(x)
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.softmax(x)

        return x

model = SimpleCNN()
model.to(device)

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=1152, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (softmax): LogSoftmax(dim=1)
)

## Training



In [3]:
import torch.optim as optim

criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Validation

In [4]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()

        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print("Training complete!")

Epoch 1, Loss: 0.4805759787579367
Epoch 2, Loss: 0.09399264111721725
Epoch 3, Loss: 0.06570091240727571
Epoch 4, Loss: 0.0527131122822783
Epoch 5, Loss: 0.04246807015445858
Training complete!


In [5]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

Accuracy on test set: 98.70%
